In [1]:
import torch
from torch import nn
from torchvision import transforms,datasets
torch.manual_seed(1)

### Data set loading

In [2]:
train_data = datasets.CIFAR10(root = './data',transform=transforms.ToTensor(),download=True)
test_data  = datasets.CIFAR10(root = './data',train=False,transform=transforms.ToTensor(),download=True)

Files already downloaded and verified
Files already downloaded and verified


In [3]:
test_data[0][0].shape

torch.Size([3, 32, 32])

### Defining batches

In [28]:
BATCH_SIZE = 500
EPOCHS = 500

train_loader = torch.utils.data.DataLoader(train_data,BATCH_SIZE,True)
test_loader = torch.utils.data.DataLoader(test_data,BATCH_SIZE)

### CNN model class

In [36]:
class CNN(nn.Module):
    def __init__(self,inp_channels=3,input_size=32,output_size=10):
        super(CNN, self).__init__()
        
        # Convolution 1
        self.cnn1 = nn.Conv2d(in_channels=inp_channels,out_channels=64,kernel_size=3)
        self.relu1 = nn.ReLU()
        
        # Convolution 2
        self.cnn2 = nn.Conv2d(in_channels=64,out_channels=64,kernel_size=3)
        self.relu2 = nn.ReLU()
                
        # Maxpool1
#         self.maxpool1 = nn.MaxPool2d(kernel_size=2)
        
        # Convolution 3
        self.cnn3 = nn.Conv2d(in_channels=64,out_channels=32,kernel_size=3)
        self.relu3 = nn.ReLU()
        
        # Maxpool 1
        self.maxpool3 = nn.MaxPool2d(kernel_size=2)
                
        # Convolution 4
        self.cnn4 = nn.Conv2d(in_channels=32,out_channels=32,kernel_size=3)
        self.relu4 = nn.ReLU()
        self.cnn4_drop = nn.Dropout2d(0.2)
        
        # Convolution 5
        self.cnn5 = nn.Conv2d(in_channels=32,out_channels=16,kernel_size=3)
        self.relu5 = nn.ReLU()
        
        # Fully Connected 1
        self.fc1 = nn.Linear(16*9*9,output_size)
#         self.relu6 = nn.ReLU()
#         self.fc2 = nn.Linear(128,output_size)
        
        

        
    def forward(self,x):
        # x = 100,3,32,32
        
        # Convolution 1
        out = self.cnn1(x) # out = 100,64,30,30
        out = self.relu1(out)
        
        # Convolution 2
        out = self.cnn2(out) # out = 100,64,28,28
        out = self.relu2(out)
        
        out = self.cnn3(out) # out = 100,32,26,26
        out = self.relu3(out)
        out = self.maxpool3(out) # out = 100,32,13,13
        
        out = self.cnn4(out) # out = 100,32,11,11
        out = self.relu4(out)
        
        out = self.cnn5(out) # out = 100,16,9,9
        out = self.relu5(out)
        
        out = out.view(out.size(0),-1)
        
        # Fully Connected layer 1
        out = self.fc1(out)
#         out = self.relu6(out)
#         out = self.fc2(out)
        
        return out

In [37]:
model = CNN()

In [38]:
if torch.cuda.is_available():
    model.cuda()
learning_rate = 0.06
 
optimizer = torch.optim.Adadelta(model.parameters(), lr=learning_rate)

In [39]:
criterion = nn.CrossEntropyLoss()

In [40]:
loss_save = torch.zeros(int(EPOCHS*len(train_data)/BATCH_SIZE))

In [41]:
def accuracy(loader,model):
    # Calculate Accuracy         
    correct = 0
    total = 0
    # Iterate through test dataset
    for images, labels in loader:
        #######################
        #  USE GPU FOR MODEL  #
        #######################
        if torch.cuda.is_available():
            images = images.cuda()

        # Forward pass only to get logits/output
        outputs = model(images)

        # Get predictions from the maximum value
        _, predicted = torch.max(outputs.data, 1)

        # Total number of labels
        total += labels.size(0)

        # Total correct predictions
        correct += (predicted.cpu() == labels.cpu()).sum().float()


    accuracy = 100. * correct / total
    return accuracy

In [ ]:
iter = 0
for epoch in range(EPOCHS):
    for i, (images, labels) in enumerate(train_loader):
         
        #######################
        #  USE GPU FOR MODEL  #
        #######################
        if torch.cuda.is_available():
            images = images.cuda()
            labels = labels.cuda()

        
        # Clear gradients w.r.t. parameters
        optimizer.zero_grad()
         
        # Forward pass to get output/logits
        outputs = model(images)
         
        # Calculate Loss: softmax --> cross entropy loss
        loss = criterion(outputs, labels)
        loss_save[iter] = loss.item()
        
        # Getting gradients w.r.t. parameters
        loss.backward()
         
        # Updating parameters
        optimizer.step()
         
        iter += 1
         
        if iter % 1000 == 0:
            # Print Loss
            test_accuracy = accuracy(test_loader,model)
            train_accuracy = accuracy(train_loader,model)
            print('Iteration: {}. Loss: {}, Test Accuracy: {},Train Accuracy: {}'.format(iter, loss.item(), test_accuracy,train_accuracy))

Iteration: 1000. Loss: 1.684850811958313, Test Accuracy: 37.5099983215332,Train Accuracy: 37.762001037597656
Iteration: 2000. Loss: 1.4713906049728394, Test Accuracy: 46.16999816894531,Train Accuracy: 45.832000732421875
Iteration: 3000. Loss: 1.417387843132019, Test Accuracy: 52.40999984741211,Train Accuracy: 53.38399887084961
Iteration: 4000. Loss: 1.3616889715194702, Test Accuracy: 54.529998779296875,Train Accuracy: 55.263999938964844
Iteration: 5000. Loss: 1.1882046461105347, Test Accuracy: 57.779998779296875,Train Accuracy: 59.61600112915039
Iteration: 6000. Loss: 1.0920581817626953, Test Accuracy: 59.2400016784668,Train Accuracy: 61.18000030517578
Iteration: 7000. Loss: 1.154478669166565, Test Accuracy: 60.970001220703125,Train Accuracy: 63.02399826049805
Iteration: 8000. Loss: 0.9769262671470642, Test Accuracy: 61.75,Train Accuracy: 64.28199768066406
Iteration: 9000. Loss: 1.0445170402526855, Test Accuracy: 63.029998779296875,Train Accuracy: 65.6520004272461
Iteration: 10000. Los

In [ ]:
loss_save.size()

In [ ]:
import seaborn as sns
from matplotlib import pyplot as plt
%matplotlib inline
ax = sns.lineplot(data = loss_save.numpy())
ax.set(xlabel = "Iterations", ylabel="Loss")
plt.show()